<a href="https://colab.research.google.com/github/ranjith-d7/app/blob/main/finetune_BLIP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will fine-tune a vision-language model (BLIP) to generate captions describing charts (bar/pie charts) with trends and summaries. BLIP (“Bootstrapping Language-Image Pre-training”) is a state-of-the-art captioning model from Salesforce.


In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import torch
from datasets import Dataset, Features, Value, Image
from transformers import BlipProcessor, BlipForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import matplotlib.pyplot as plt
from PIL import Image as PILImage


In [ ]:


charts = []
captions = []
# Example 1: Bar chart (ascending)
plt.figure(figsize=(4,3))
x = ['Q1','Q2','Q3','Q4']
y = [1, 3, 6, 9]
plt.bar(x, y, color='skyblue')
plt.title("Sales by Quarter")
plt.ylabel("Sales")
plt.savefig('bar_up.png')
plt.close()
charts.append('bar_up.png')
captions.append("Bar chart showing steadily increasing sales from Q1 to Q4.")

# Example 2: Bar chart (descending)
plt.figure(figsize=(4,3))
x = ['2016','2017','2018','2019']
y = [9, 6, 3, 1]
plt.bar(x, y, color='lightgreen')
plt.title("Downloads by Year")
plt.ylabel("Number of downloads (millions)")
plt.savefig('bar_down.png')
plt.close()
charts.append('bar_down.png')
captions.append("Bar chart showing decreasing downloads from 2016 to 2019.")

# Example 3: Pie chart
plt.figure(figsize=(4,3))
sizes = [50, 25, 15, 10]
labels = ['Category A','Category B','Category C','Category D']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title("Market Share Distribution")
plt.savefig('pie1.png')
plt.close()
charts.append('pie1.png')
captions.append("Pie chart showing market share, with Category A as the largest slice.")

# Example 4: Pie chart (different)
plt.figure(figsize=(4,3))
sizes = [40, 30, 20, 10]
labels = ['Red','Blue','Green','Yellow']
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.title("Color Distribution")
plt.savefig('pie2.png')
plt.close()
charts.append('pie2.png')
captions.append("Pie chart showing distribution of colors: Red and Blue are the largest segments.")

# Assemble into a dataset (features: image and text)
features = Features({"image": Image(decode=True), "text": Value("string")})
data = {"image": charts, "text": captions}
dataset = Dataset.from_dict(data, features=features)
# Split into train and test
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


In [ ]:
import torch
from datasets import Dataset, Features, Value, Image
from transformers import BlipProcessor, BlipForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import matplotlib.pyplot as plt
from PIL import Image as PILImage


In [ ]:
from transformers import BlipProcessor

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
prefix = "A chart showing"

def preprocess_function(examples):
    # examples["image"] is already a PIL Image, examples["text"] is the caption
    encoding = processor(
        images=examples["image"],
        text=[prefix] * len(examples["image"]),
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="np"
    )
    # copy input_ids → labels so the model computes loss
    encoding["labels"] = encoding["input_ids"].copy()
    return encoding

# apply to both train and eval
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["image","text"])
eval_dataset  = eval_dataset.map (preprocess_function, batched=True, remove_columns=["image","text"])



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=processor.tokenizer, model=model, label_pad_token_id=-100)
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    remove_unused_columns=False # Instead, pass it to TrainingArguments
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=processor.tokenizer
)


<ipython-input-8-15e5ca48d676>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ranjithdhanasekaran (ranjithdhanasekaran-b-s-abdir-rahman-crescent-institute-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=6, training_loss=10.568430582682291, metrics={'train_runtime': 154.0097, 'train_samples_per_second': 0.058, 'train_steps_per_second': 0.039, 'total_flos': 5340812870025216.0, 'train_loss': 10.568430582682291, 'epoch': 3.0})

In [ ]:
test_image = PILImage.open('bar_up.png')
inputs = processor(images=test_image, text="A chart showing", return_tensors="pt")

# Move the input tensors to the same device as the model's weights
# Assuming your model is on a CUDA device, if not, replace 'cuda' with 'cpu'
inputs = inputs.to(model.device)

outputs = model.generate(pixel_values=inputs.pixel_values)
caption = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated caption:", caption)



Generated caption: a chart showing the number of cases of the coronavirus in the us
